In [1]:
from pipeline_raw2 import *

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from catboost import CatBoostClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import set_config

np.random.seed(101)
pd.set_option('display.max_columns', None)
set_config(transform_output="pandas")

In [3]:
df = pd.read_parquet(r'C:\Users\minht\Desktop\MLOps\data\phase-2\prob-2\raw_train.parquet')

In [4]:
df = df.sample(frac=1)

In [5]:
df.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,label
34187,0.081963,tcp,-,FIN,22.0,14.0,1470.0,1728.0,31.0,29.0,1.370374e+05,156656.046900,5.0,4.0,3.903000,6.248692,399.950226,14.982982,255.0,1.213830e+09,3.363719e+09,255.0,0.000707,0.000582,0.000125,67.0,123.0,0.0,0.0,9.0,0.0,4.0,1.0,1.0,3.0,0.0,0.0,0.0,3.0,16.0,0.0,Normal
28653,0.000010,udp,dns,INT,2.0,0.0,114.0,0.0,254.0,0.0,4.560000e+07,0.000000,0.0,0.0,0.010000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,57.0,0.0,0.0,0.0,4.0,2.0,5.0,4.0,4.0,4.0,0.0,0.0,0.0,4.0,4.0,0.0,Other
7570,0.796955,tcp,-,FIN,10.0,10.0,608.0,646.0,254.0,252.0,5.500938e+03,5842.237305,2.0,3.0,88.550556,80.579555,4858.346581,127.869945,255.0,2.375786e+09,2.121203e+09,255.0,0.159596,0.070978,0.088618,61.0,65.0,0.0,0.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,Exploits
50165,0.000009,udp,dns,INT,2.0,0.0,114.0,0.0,254.0,0.0,5.066666e+07,0.000000,0.0,0.0,0.009000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,57.0,0.0,0.0,0.0,33.0,2.0,26.0,26.0,13.0,33.0,0.0,0.0,0.0,26.0,33.0,0.0,Other
55555,0.376387,tcp,-,FIN,10.0,6.0,1612.0,268.0,254.0,252.0,3.084060e+04,4761.057129,2.0,1.0,38.067556,59.815801,2007.245167,88.242922,255.0,1.663055e+09,3.271902e+08,255.0,0.136073,0.077299,0.058774,161.0,45.0,0.0,0.0,7.0,1.0,2.0,2.0,1.0,7.0,0.0,0.0,0.0,2.0,7.0,0.0,Denial of Service


In [6]:
nums = [
    "feature1",    "feature5",    "feature6",    "feature7",    "feature8",    "feature9",    "feature10",    "feature11",
    "feature12",    "feature13",    "feature14",    "feature15",    "feature16",    "feature17",    "feature18",    "feature19",
    "feature20",    "feature21",    "feature22",    "feature23",    "feature24",    "feature25",    "feature26",    "feature27",
    "feature28",    "feature29",    "feature30",    "feature31",    "feature32",    "feature33",    "feature34",    "feature35",
    "feature36",    "feature37",    "feature38",    "feature39",    "feature40",    "feature41"
]

cats = [
    "feature2", "feature3", "feature4"
]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['label']), df['label'], test_size=0.2, random_state=101)

In [8]:
VARIABLES_COLUMNS_REMOVE_IN_STEP1_REDUNDANCE = []

dup_variance_pipeline = Pipeline(
    steps=[
        ('low_variance_and_dup', DuplicatedReduction(nums)),
        ('remove_missing_columns', MissingRemoving())
    ]
)


numerical_pipeline = Pipeline(
    steps=[
        ('num_selector', FeatureSelector(nums)),
        ('outlier_detect', OutlierDetector(nums))
    ]
)

categorical_pipeline = Pipeline(
    steps=[
        ('cat_selector', FeatureSelector(cats))
    ]
)

feat_union = PandasFeatureUnion([
    ('num_pipeline', numerical_pipeline),
    ('cat_pipeline', categorical_pipeline)
])


binning_process_class = WoeImputing(cats=cats, nums=nums)

binning_process = Pipeline(
    steps=[
        ('woe_binning', binning_process_class)
    ]
)

remove_corr_process = RemoveHighCorr()
 

clf = Pipeline(
    steps=[
        ('duplowvarprocesser', dup_variance_pipeline),
        ('preprocesser', feat_union),
        ('woeprocessor', binning_process)
    ]
)


In [10]:
from optbinning import OptimalBinning, MulticlassOptimalBinning

In [ ]:
s = MulticlassOptimalBinning()
k = OptimalBinning()

In [12]:
# clf.fit(X_train[nums+cats], y_train)

In [ ]:
X_train = clf.transform(X_train)
X_test = clf.transform(X_test)

In [ ]:
X_train.shape

In [13]:
X_train.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41
29818,0.000004,udp,dns,INT,2.0,0.0,114.0,0.0,254.0,0.0,1.140000e+08,0.000000,0.0,0.0,0.004,0.000,0.000000,0.0000,0.0,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,57.0,0.0,0.0,0.0,33.0,2.0,33.0,33.0,16.0,33.0,0.0,0.0,0.0,33.0,33.0,0.0
34436,0.000006,udp,dns,INT,2.0,0.0,114.0,0.0,254.0,0.0,7.600000e+07,0.000000,0.0,0.0,0.006,0.000,0.000000,0.0000,0.0,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,57.0,0.0,0.0,0.0,19.0,2.0,17.0,17.0,17.0,19.0,0.0,0.0,0.0,17.0,19.0,0.0
15355,0.000011,udp,dns,INT,2.0,0.0,114.0,0.0,254.0,0.0,4.145454e+07,0.000000,0.0,0.0,0.011,0.000,0.000000,0.0000,0.0,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,57.0,0.0,0.0,0.0,14.0,2.0,7.0,7.0,3.0,7.0,0.0,0.0,0.0,10.0,14.0,0.0
48909,0.000009,udp,dns,INT,2.0,0.0,114.0,0.0,254.0,0.0,5.066666e+07,0.000000,0.0,0.0,0.009,0.000,0.000000,0.0000,0.0,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,57.0,0.0,0.0,0.0,34.0,2.0,16.0,16.0,16.0,34.0,0.0,0.0,0.0,16.0,34.0,0.0
32074,1.320121,tcp,http,FIN,10.0,8.0,800.0,354.0,254.0,252.0,4.363236e+03,1878.615601,2.0,1.0,141.116,175.283,8139.509486,346.8415,255.0,2.646710e+09,978994462.0,255.0,0.203945,0.093138,0.110807,80.0,44.0,1.0,0.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0


In [16]:
model = CatBoostClassifier(
    iterations=2000,
    random_seed=101,
    learning_rate=0.05,
    eval_metric='AUC'
)

In [17]:
model.fit(X_train, y_train, eval_set=(X_test, y_test), cat_features=cats)

0:	test: 0.8774929	best: 0.8774929 (0)	total: 115ms	remaining: 3m 50s
1:	test: 0.9017191	best: 0.9017191 (1)	total: 230ms	remaining: 3m 49s
2:	test: 0.9039604	best: 0.9039604 (2)	total: 341ms	remaining: 3m 47s
3:	test: 0.9037137	best: 0.9039604 (2)	total: 464ms	remaining: 3m 51s
4:	test: 0.9051714	best: 0.9051714 (4)	total: 579ms	remaining: 3m 51s
5:	test: 0.9088451	best: 0.9088451 (5)	total: 694ms	remaining: 3m 50s
6:	test: 0.9093833	best: 0.9093833 (6)	total: 807ms	remaining: 3m 49s
7:	test: 0.9132366	best: 0.9132366 (7)	total: 924ms	remaining: 3m 50s
8:	test: 0.9149065	best: 0.9149065 (8)	total: 1.04s	remaining: 3m 50s
9:	test: 0.9152428	best: 0.9152428 (9)	total: 1.17s	remaining: 3m 51s
10:	test: 0.9159800	best: 0.9159800 (10)	total: 1.29s	remaining: 3m 53s
11:	test: 0.9162994	best: 0.9162994 (11)	total: 1.42s	remaining: 3m 54s
12:	test: 0.9169194	best: 0.9169194 (12)	total: 1.53s	remaining: 3m 53s
13:	test: 0.9183759	best: 0.9183759 (13)	total: 1.65s	remaining: 3m 54s
14:	test: 0.

114:	test: 0.9469104	best: 0.9469104 (114)	total: 13.6s	remaining: 3m 43s
115:	test: 0.9469902	best: 0.9469902 (115)	total: 13.7s	remaining: 3m 43s
116:	test: 0.9471327	best: 0.9471327 (116)	total: 13.9s	remaining: 3m 43s
117:	test: 0.9471456	best: 0.9471456 (117)	total: 14s	remaining: 3m 42s
118:	test: 0.9473990	best: 0.9473990 (118)	total: 14.1s	remaining: 3m 42s
119:	test: 0.9474073	best: 0.9474073 (119)	total: 14.2s	remaining: 3m 42s
120:	test: 0.9474762	best: 0.9474762 (120)	total: 14.3s	remaining: 3m 42s
121:	test: 0.9477760	best: 0.9477760 (121)	total: 14.4s	remaining: 3m 42s
122:	test: 0.9478075	best: 0.9478075 (122)	total: 14.5s	remaining: 3m 41s
123:	test: 0.9478829	best: 0.9478829 (123)	total: 14.6s	remaining: 3m 41s
124:	test: 0.9478531	best: 0.9478829 (123)	total: 14.8s	remaining: 3m 41s
125:	test: 0.9479479	best: 0.9479479 (125)	total: 14.9s	remaining: 3m 41s
126:	test: 0.9479682	best: 0.9479682 (126)	total: 15s	remaining: 3m 41s
127:	test: 0.9480930	best: 0.9480930 (127)

226:	test: 0.9514582	best: 0.9514582 (226)	total: 26.5s	remaining: 3m 27s
227:	test: 0.9514795	best: 0.9514795 (227)	total: 26.6s	remaining: 3m 27s
228:	test: 0.9514575	best: 0.9514795 (227)	total: 26.8s	remaining: 3m 26s
229:	test: 0.9514756	best: 0.9514795 (227)	total: 26.9s	remaining: 3m 26s
230:	test: 0.9515365	best: 0.9515365 (230)	total: 27s	remaining: 3m 26s
231:	test: 0.9515319	best: 0.9515365 (230)	total: 27.1s	remaining: 3m 26s
232:	test: 0.9515370	best: 0.9515370 (232)	total: 27.2s	remaining: 3m 26s
233:	test: 0.9515371	best: 0.9515371 (233)	total: 27.3s	remaining: 3m 26s
234:	test: 0.9515330	best: 0.9515371 (233)	total: 27.4s	remaining: 3m 25s
235:	test: 0.9514495	best: 0.9515371 (233)	total: 27.5s	remaining: 3m 25s
236:	test: 0.9514769	best: 0.9515371 (233)	total: 27.7s	remaining: 3m 25s
237:	test: 0.9514304	best: 0.9515371 (233)	total: 27.8s	remaining: 3m 25s
238:	test: 0.9514628	best: 0.9515371 (233)	total: 27.9s	remaining: 3m 25s
239:	test: 0.9514850	best: 0.9515371 (23

338:	test: 0.9524126	best: 0.9524644 (336)	total: 39.2s	remaining: 3m 12s
339:	test: 0.9523908	best: 0.9524644 (336)	total: 39.3s	remaining: 3m 11s
340:	test: 0.9523684	best: 0.9524644 (336)	total: 39.4s	remaining: 3m 11s
341:	test: 0.9523761	best: 0.9524644 (336)	total: 39.5s	remaining: 3m 11s
342:	test: 0.9524267	best: 0.9524644 (336)	total: 39.6s	remaining: 3m 11s
343:	test: 0.9524387	best: 0.9524644 (336)	total: 39.8s	remaining: 3m 11s
344:	test: 0.9524120	best: 0.9524644 (336)	total: 39.9s	remaining: 3m 11s
345:	test: 0.9523980	best: 0.9524644 (336)	total: 40s	remaining: 3m 11s
346:	test: 0.9524022	best: 0.9524644 (336)	total: 40.1s	remaining: 3m 10s
347:	test: 0.9524047	best: 0.9524644 (336)	total: 40.2s	remaining: 3m 10s
348:	test: 0.9523949	best: 0.9524644 (336)	total: 40.3s	remaining: 3m 10s
349:	test: 0.9523968	best: 0.9524644 (336)	total: 40.4s	remaining: 3m 10s
350:	test: 0.9523917	best: 0.9524644 (336)	total: 40.5s	remaining: 3m 10s
351:	test: 0.9523906	best: 0.9524644 (33

451:	test: 0.9529026	best: 0.9529026 (451)	total: 51.9s	remaining: 2m 57s
452:	test: 0.9529040	best: 0.9529040 (452)	total: 52s	remaining: 2m 57s
453:	test: 0.9529361	best: 0.9529361 (453)	total: 52.1s	remaining: 2m 57s
454:	test: 0.9529426	best: 0.9529426 (454)	total: 52.2s	remaining: 2m 57s
455:	test: 0.9529626	best: 0.9529626 (455)	total: 52.3s	remaining: 2m 57s
456:	test: 0.9528898	best: 0.9529626 (455)	total: 52.4s	remaining: 2m 57s
457:	test: 0.9528882	best: 0.9529626 (455)	total: 52.5s	remaining: 2m 56s
458:	test: 0.9529001	best: 0.9529626 (455)	total: 52.7s	remaining: 2m 56s
459:	test: 0.9529183	best: 0.9529626 (455)	total: 52.8s	remaining: 2m 56s
460:	test: 0.9529394	best: 0.9529626 (455)	total: 52.9s	remaining: 2m 56s
461:	test: 0.9529423	best: 0.9529626 (455)	total: 53s	remaining: 2m 56s
462:	test: 0.9529563	best: 0.9529626 (455)	total: 53.1s	remaining: 2m 56s
463:	test: 0.9529642	best: 0.9529642 (463)	total: 53.2s	remaining: 2m 56s
464:	test: 0.9529359	best: 0.9529642 (463)

...

1446:	test: 0.9527931	best: 0.9535138 (931)	total: 2m 48s	remaining: 1m 4s
1447:	test: 0.9527802	best: 0.9535138 (931)	total: 2m 48s	remaining: 1m 4s
1448:	test: 0.9527801	best: 0.9535138 (931)	total: 2m 48s	remaining: 1m 4s
1449:	test: 0.9527799	best: 0.9535138 (931)	total: 2m 48s	remaining: 1m 4s
1450:	test: 0.9527623	best: 0.9535138 (931)	total: 2m 48s	remaining: 1m 3s
1451:	test: 0.9527618	best: 0.9535138 (931)	total: 2m 48s	remaining: 1m 3s
1452:	test: 0.9527564	best: 0.9535138 (931)	total: 2m 49s	remaining: 1m 3s
1453:	test: 0.9527612	best: 0.9535138 (931)	total: 2m 49s	remaining: 1m 3s
1454:	test: 0.9527465	best: 0.9535138 (931)	total: 2m 49s	remaining: 1m 3s
1455:	test: 0.9527478	best: 0.9535138 (931)	total: 2m 49s	remaining: 1m 3s
1456:	test: 0.9527420	best: 0.9535138 (931)	total: 2m 49s	remaining: 1m 3s
1457:	test: 0.9527558	best: 0.9535138 (931)	total: 2m 49s	remaining: 1m 3s
1458:	test: 0.9527603	best: 0.9535138 (931)	total: 2m 49s	remaining: 1m 2s
1459:	test: 0.9527507	bes

1557:	test: 0.9527482	best: 0.9535138 (931)	total: 3m	remaining: 51.3s
1558:	test: 0.9527568	best: 0.9535138 (931)	total: 3m 1s	remaining: 51.2s
1559:	test: 0.9527678	best: 0.9535138 (931)	total: 3m 1s	remaining: 51.1s
1560:	test: 0.9527475	best: 0.9535138 (931)	total: 3m 1s	remaining: 51s
1561:	test: 0.9527583	best: 0.9535138 (931)	total: 3m 1s	remaining: 50.9s
1562:	test: 0.9527522	best: 0.9535138 (931)	total: 3m 1s	remaining: 50.8s
1563:	test: 0.9527621	best: 0.9535138 (931)	total: 3m 1s	remaining: 50.6s
1564:	test: 0.9527565	best: 0.9535138 (931)	total: 3m 1s	remaining: 50.5s
1565:	test: 0.9527605	best: 0.9535138 (931)	total: 3m 1s	remaining: 50.4s
1566:	test: 0.9527599	best: 0.9535138 (931)	total: 3m 1s	remaining: 50.3s
1567:	test: 0.9527733	best: 0.9535138 (931)	total: 3m 2s	remaining: 50.2s
1568:	test: 0.9527711	best: 0.9535138 (931)	total: 3m 2s	remaining: 50s
1569:	test: 0.9527812	best: 0.9535138 (931)	total: 3m 2s	remaining: 49.9s
1570:	test: 0.9527782	best: 0.9535138 (931)	t

1668:	test: 0.9527055	best: 0.9535138 (931)	total: 3m 13s	remaining: 38.3s
1669:	test: 0.9527028	best: 0.9535138 (931)	total: 3m 13s	remaining: 38.2s
1670:	test: 0.9527012	best: 0.9535138 (931)	total: 3m 13s	remaining: 38.1s
1671:	test: 0.9526987	best: 0.9535138 (931)	total: 3m 13s	remaining: 38s
1672:	test: 0.9527023	best: 0.9535138 (931)	total: 3m 13s	remaining: 37.9s
1673:	test: 0.9527016	best: 0.9535138 (931)	total: 3m 13s	remaining: 37.8s
1674:	test: 0.9526982	best: 0.9535138 (931)	total: 3m 13s	remaining: 37.6s
1675:	test: 0.9527149	best: 0.9535138 (931)	total: 3m 14s	remaining: 37.5s
1676:	test: 0.9527194	best: 0.9535138 (931)	total: 3m 14s	remaining: 37.4s
1677:	test: 0.9527089	best: 0.9535138 (931)	total: 3m 14s	remaining: 37.3s
1678:	test: 0.9527125	best: 0.9535138 (931)	total: 3m 14s	remaining: 37.2s
1679:	test: 0.9527092	best: 0.9535138 (931)	total: 3m 14s	remaining: 37.1s
1680:	test: 0.9527076	best: 0.9535138 (931)	total: 3m 14s	remaining: 36.9s
1681:	test: 0.9527096	best:

1779:	test: 0.9527906	best: 0.9535138 (931)	total: 3m 25s	remaining: 25.4s
1780:	test: 0.9527930	best: 0.9535138 (931)	total: 3m 25s	remaining: 25.3s
1781:	test: 0.9527979	best: 0.9535138 (931)	total: 3m 25s	remaining: 25.2s
1782:	test: 0.9527974	best: 0.9535138 (931)	total: 3m 26s	remaining: 25.1s
1783:	test: 0.9527997	best: 0.9535138 (931)	total: 3m 26s	remaining: 25s
1784:	test: 0.9527902	best: 0.9535138 (931)	total: 3m 26s	remaining: 24.8s
1785:	test: 0.9527950	best: 0.9535138 (931)	total: 3m 26s	remaining: 24.7s
1786:	test: 0.9527829	best: 0.9535138 (931)	total: 3m 26s	remaining: 24.6s
1787:	test: 0.9527827	best: 0.9535138 (931)	total: 3m 26s	remaining: 24.5s
1788:	test: 0.9527814	best: 0.9535138 (931)	total: 3m 26s	remaining: 24.4s
1789:	test: 0.9527870	best: 0.9535138 (931)	total: 3m 26s	remaining: 24.3s
1790:	test: 0.9527927	best: 0.9535138 (931)	total: 3m 26s	remaining: 24.1s
1791:	test: 0.9527889	best: 0.9535138 (931)	total: 3m 26s	remaining: 24s
1792:	test: 0.9527922	best: 0

1890:	test: 0.9526605	best: 0.9535138 (931)	total: 3m 38s	remaining: 12.6s
1891:	test: 0.9526568	best: 0.9535138 (931)	total: 3m 38s	remaining: 12.5s
1892:	test: 0.9526599	best: 0.9535138 (931)	total: 3m 38s	remaining: 12.4s
1893:	test: 0.9526819	best: 0.9535138 (931)	total: 3m 38s	remaining: 12.2s
1894:	test: 0.9526833	best: 0.9535138 (931)	total: 3m 38s	remaining: 12.1s
1895:	test: 0.9526792	best: 0.9535138 (931)	total: 3m 39s	remaining: 12s
1896:	test: 0.9526742	best: 0.9535138 (931)	total: 3m 39s	remaining: 11.9s
1897:	test: 0.9526665	best: 0.9535138 (931)	total: 3m 39s	remaining: 11.8s
1898:	test: 0.9526748	best: 0.9535138 (931)	total: 3m 39s	remaining: 11.7s
1899:	test: 0.9526736	best: 0.9535138 (931)	total: 3m 39s	remaining: 11.6s
1900:	test: 0.9526746	best: 0.9535138 (931)	total: 3m 39s	remaining: 11.4s
1901:	test: 0.9526791	best: 0.9535138 (931)	total: 3m 39s	remaining: 11.3s
1902:	test: 0.9526851	best: 0.9535138 (931)	total: 3m 39s	remaining: 11.2s
1903:	test: 0.9526815	best:

In [18]:
vl = pd.DataFrame([X_train.columns, model.get_feature_importance()]).T
vl.columns = ['Feature', 'Importance']
vl = vl.sort_values('Importance', ascending=False).reset_index(drop=True)
vl

,Feature,Importance
0,feature7,14.592646
1,feature9,12.465321
2,feature26,11.565155
3,feature3,10.164494
4,feature2,7.307586
5,feature35,5.837536
6,feature40,5.642962
7,feature8,3.93079
8,feature27,3.048274
9,feature30,2.525443


In [20]:
vl['Feature'].values[:21]

array(['feature7', 'feature9', 'feature26', 'feature3', 'feature2',
       'feature35', 'feature40', 'feature8', 'feature27', 'feature30',
       'feature34', 'feature23', 'feature12', 'feature11', 'feature39',
       'feature1', 'feature33', 'feature10', 'feature4', 'feature31',
       'feature15'], dtype=object)

In [ ]:
import pickle

with open(r'C:\Users\minht\Desktop\MLOps\src\phase1\models\pipeline.pkl', 'wb') as f:
    pickle.dump(s, f)

In [ ]:
vl = pd.DataFrame([X_train.columns, model.get_feature_importance()]).T
vl.columns = ['Feature', 'Importance']
vl = vl.sort_values('Importance', ascending=False)
vl

In [ ]:
import pickle

with open(r'C:\Users\minht\Desktop\MLOps\src\phase1\models\problem1.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
cc = X_train.head(100)

In [ ]:
%time
tmp = model.predict_proba(X_train)[:, 1]

In [ ]:
tmp = model.predict_proba(X_train)[:, 1]
tmp = pd.DataFrame(tmp, columns=['Pro'])

In [ ]:
tmp

In [ ]:
tmp['Range'], _ = pd.qcut(tmp['Pro'], 10, retbins=True)

In [ ]:
tmp['Range'].value_counts(normalize=True).sort_index()

In [ ]:
_

In [ ]:
import statsmodels.api as sm

In [ ]:
logit_model = sm.Logit(y_train.values, X_train.values)


result = logit_model.fit()

print(result.summary())


In [ ]:
pred = result.predict(X_val.values)

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_val, pred)